# Homework Starter — Stage 05: Data Storage
Name: 
Date: 

Objectives:
- Env-driven paths to `data/raw/` and `data/processed/`
- Save CSV and Parquet; reload and validate
- Abstract IO with utility functions; document choices

In [1]:
import os, pathlib, datetime as dt
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
RAW = pathlib.Path(os.getenv('DATA_DIR_RAW', 'data/raw'))
PROC = pathlib.Path(os.getenv('DATA_DIR_PROCESSED', 'data/processed'))
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)
print('RAW ->', RAW.resolve())
print('PROC ->', PROC.resolve())

RAW -> /Users/aditya/Documents/bootcamp_aditya_shah/homework/homework5/data/raw
PROC -> /Users/aditya/Documents/bootcamp_aditya_shah/homework/homework5/data/processed


## 1) Create or Load a Sample DataFrame
You may reuse data from prior stages or create a small synthetic dataset.

In [5]:

import zipfile
from pathlib import Path

files = ["/Users/aditya/Documents/USDZAR-2025-07.csv"]

with zipfile.ZipFile("/Users/aditya/Documents/bootcamp_aditya_shah/homework/homework5/data/USD_ZAR_2025_07.zip", "w", zipfile.ZIP_DEFLATED) as z:
    for f in files:
        z.write(f, arcname=Path(f).name)


In [6]:
import numpy as np
import pandas as pd

df = pd.read_csv("/Users/aditya/Documents/bootcamp_aditya_shah/homework/homework5/data/USD_ZAR_2025_07.zip")
print(df.head())


  exchange              date_time      bid      ask
0  USD/ZAR  20250701 00:00:00.141  17.7123  17.7338
1  USD/ZAR  20250701 00:00:00.156  17.7167  17.7267
2  USD/ZAR  20250701 00:00:00.234  17.7185  17.7267
3  USD/ZAR  20250701 00:00:00.234  17.7185  17.7272
4  USD/ZAR  20250701 00:00:00.313  17.7139  17.7336


## 2) Save CSV to data/raw/ and Parquet to data/processed/ (TODO)
- Use timestamped filenames.
- Handle missing Parquet engine gracefully.

In [3]:
# !pip install pyarrow fastparquet

In [4]:

# TODO: Save CSV
csv_path = RAW / "USD_ZAR_2025_07.csv"
df.to_csv(csv_path, index=False)
csv_path

# TODO: Save Parquet
pq_path = PROC / f"USD_ZAR_2025_07.parquet"
try:
    df.to_parquet(pq_path)
except Exception as e:
    print('Parquet engine not available. Install pyarrow or fastparquet to complete this step.')
    pq_path = None
pq_path

PosixPath('/Users/aditya/Documents/bootcamp_aditya_shah/homework/homework5/data/processed/USD_ZAR_2025_07.parquet')

## 3) Reload and Validate (TODO)
- Compare shapes and key dtypes.

In [8]:


def validate_loaded(original: pd.DataFrame, reloaded: pd.DataFrame) -> dict:
    """
    Validate that the reloaded DataFrame matches expected structure and dtypes.
    """
    checks = {
        # Shape check
        'shape_equal': original.shape == reloaded.shape,
        
        # Column checks
        'has_columns': list(reloaded.columns) == ["exchange", "date_time", "bid", "ask"],
        
        # Type checks
        'exchange_is_string': pd.api.types.is_string_dtype(reloaded['exchange']) if 'exchange' in reloaded.columns else False,
        'date_time_is_datetime': pd.api.types.is_datetime64_any_dtype(reloaded['date_time']) if 'date_time' in reloaded.columns else False,
        'bid_is_numeric': pd.api.types.is_numeric_dtype(reloaded['bid']) if 'bid' in reloaded.columns else False,
        'ask_is_numeric': pd.api.types.is_numeric_dtype(reloaded['ask']) if 'ask' in reloaded.columns else False,
    }
    return checks


In [9]:
if pq_path:
    try:
        df_pq = pd.read_parquet(pq_path)
        validate_loaded(df, df_pq)
    except Exception as e:
        print('Parquet read failed:', e)

## 4) Utilities (TODO)
- Implement `detect_format`, `write_df`, `read_df`.
- Use suffix to route; create parent dirs if needed; friendly errors for Parquet.

In [ ]:
import typing as t, pathlib

def detect_format(path: t.Union[str, pathlib.Path]):
    s = str(path).lower()
    if s.endswith('.csv'): return 'csv'
    if s.endswith('.parquet') or s.endswith('.pq') or s.endswith('.parq'): return 'parquet'
    raise ValueError('Unsupported format: ' + s)

def write_df(df: pd.DataFrame, path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path); p.parent.mkdir(parents=True, exist_ok=True)
    fmt = detect_format(p)
    if fmt == 'csv':
        df.to_csv(p, index=False)
    else:
        try:
            df.to_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e
    return p

def read_df(path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path)
    fmt = detect_format(p)
    if fmt == 'csv':
        return pd.read_csv(p, parse_dates=['date']) if 'date' in pd.read_csv(p, nrows=0).columns else pd.read_csv(p)
    else:
        try:
            return pd.read_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e

def ts(): return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

# Demo
p_csv = RAW / f"util_{ts()}.csv"
p_pq  = PROC / f"util_{ts()}.parquet"
write_df(df, p_csv); read_df(p_csv).head()
try:
    write_df(df, p_pq)
    read_df(p_pq).head()
except RuntimeError as e:
    print('Skipping Parquet util demo:', e)

## 5) Documentation (TODO)
- Update README with a **Data Storage** section (folders, formats, env usage).
- Summarize validation checks and any assumptions.